In [1]:
import time
from urllib import FancyURLopener  # This is library that helps us create the headless browser
from random import choice #This library helps pick a random item from a list
import bs4 as bs
import datetime
from datetime import timedelta, date
import pandas as pd
from selenium import webdriver
import sys
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [2]:
user_agents = [
    'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36',
    'Opera/9.80 (X11; Linux i686; Ubuntu/14.10) Presto/2.12.388 Version/12.16',
    'Mozilla/5.0 (Windows; U; Windows NT 6.1; rv:2.2) Gecko/20110201',
    'Mozilla/5.0 (Macintosh; Intel MaPc OS X 10_9_3) AppleWebKit/537.75.14 (KHTML, like Gecko) Version/7.0.3 Safari/7046A194A',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246'
]

class MyOpener(FancyURLopener, object):
    version = choice(user_agents)

# Get each article and their link
def getArticleMeta(infotable):
    for i in range(0,len(infotable)):
        CallDesc.append(infotable[i].getText().encode('utf-8'))
        CallLink.append(infotable[i]['href'])
        
def getDate(i):
    date = i.find("span", class_= "date")
    date = date.getText()
    try:
        date = pd.to_datetime(date,infer_datetime_format=True)
    except ValueError:
        if "Yesterday" in date:
            Date = datetime.date.today() - timedelta(1)
            date = date[10:]
            date = str(Date) + date
        else:
            date = date.split(',')[1]
            date = date + ' ' + str(2017)
        date = pd.to_datetime(date,infer_datetime_format=True)
    return date

def getTicker(companyName):
    df = pd.read_csv('company.txt', sep = '\t')
    df = df[df['Country'] == 'USA']
    dfD = df.set_index('Name')['Ticker'].to_dict()
    del df
    return dfD[process.extractOne(companyName, dfD.keys())[0]]

In [3]:
# This uses the search tool on Seeking Alpha to get pull specific company's transcript.

def getTranscriptBySearch(companyName):
    CallDesc = []
    CallLink = []
    company = companyName
    driver = webdriver.Firefox(executable_path='/Users/sumerdhillon/Dropbox/MS-BAIM/Second Module/MGMT 590/Lecture 7/geckodriver')
    try:
        ticker = getTicker(company)
        url = 'https://seekingalpha.com/search/transcripts?term='+ ticker
        driver.get(url) # Start the browser and open 'url'
    except KeyError:
        print "That company doesn't exist in my masterlist"
        raise

    assert 'Seeking Alpha' in driver.title # Wait for the page to load
    html = driver.page_source # Get the html of the page
    driver.quit() # Close the browser

    soup = bs.BeautifulSoup(html, 'html.parser')

    infotable = soup.find_all("div", class_= "transcript_link")
    for i in infotable:
        date = getDate(i)
        if (date >= datetime.datetime(2016,9,1)) & (ticker in i.find('a').getText()):
            CallDesc.append(i.find('a').getText().encode('utf-8'))
            CallLink.append(i.find('a')['href'])
    CreateTextFiles(CallLink)

In [4]:
# This goes into the transcript link and extracts the text from each link
def CreateTextFiles(CallLink):
    for link in range(0,len(CallLink)):
        myopener = MyOpener()
        page=myopener.open('https://seekingalpha.com' + CallLink[link])

        html = page.read()
        soup = bs.BeautifulSoup(html, 'lxml')

        infotable = soup.find_all("p", class_= "p")
        try:
            Output_File = open(str(infotable[0].find("a")['title']) + 
                               str(infotable[2].getText()) + ".txt",'w')
        except IndexError as exc:
            continue
        except TypeError as typ:
            continue

        for i in range(0,len(infotable)):
            if(infotable[i].getText().encode('utf-8').startswith('Copyright policy:')):
                break
            if(len(infotable[i].getText()) == 0):
                time.sleep(600)
                continue
            Output_File.write(infotable[i].getText().encode('utf-8'))
            Output_File.write("\n")
        Output_File.close()

In [17]:
df = pd.read_csv('AdvanceCapital.csv', usecols = ['company_name'])
companies = df['company_name'].tolist()
for company in companies:
    counter = 0
    getTranscriptBySearch(company)

In [18]:
getTranscriptBySearch('Whirlpool')

In [ ]:
# This pulls data from 1 single page of the earnings call transcript
CallDesc = []
CallLink = []

myopener = MyOpener()
page=myopener.open('https://seekingalpha.com/earnings/earnings-call-transcripts')

html = page.read()
soup = bs.BeautifulSoup(html, 'lxml')

infotable = soup.find_all("a", class_= "dashboard-article-link")
for i in range(0,len(infotable)):
    CallDesc.append(infotable[i].getText().encode('ascii'))
    CallLink.append(infotable[i]['href'])

In [ ]:
# This counts the number of pages of transcripts there are on the website.
myopener = MyOpener()
page=myopener.open('https://seekingalpha.com/earnings/earnings-call-transcripts')

html = page.read()
soup = bs.BeautifulSoup(html, 'lxml')

dots = soup.find("li", class_= "dots")
number = dots.find_next_siblings("li")
number = number[0].find("a").getText()

print number

In [ ]:
# This cell tries to go through each page and do the same as before(get the links and names)
# Note: There is a chance of the website detecting and stopping you after a while

CallDesc = []
CallLink = []
counter = 0
        
# Open the site for the first time
myopener = MyOpener()
page=myopener.open('https://seekingalpha.com/earnings/earnings-call-transcripts')
html = page.read()
soup = bs.BeautifulSoup(html, 'lxml')

# Find the total number of pages on the site at this moment
dots = soup.find("li", class_= "dots")
number = dots.find_next_siblings("li")
number = number[0].find("a").getText()

# Now loop through each page
for i in range(0,int(number)):
    myopener = MyOpener()
    page=myopener.open('https://seekingalpha.com/earnings/earnings-call-transcripts/' + str(i+1))
    html = page.read()
    soup = bs.BeautifulSoup(html, 'lxml')
    
    counter += 1
    infotable = soup.find_all("a", class_= "dashboard-article-link")
    print i+1, len(infotable)
    getArticleMeta(infotable)
    time.sleep(choice(range(5,10)))
    
    if counter == 100:
        break

In [ ]:
CallDesc = []
CallLink = []

myopener = MyOpener()
page=myopener.open('https://seekingalpha.com/earnings/earnings-call-transcripts/2')

html = page.read()
soup = bs.BeautifulSoup(html, 'lxml')


infotable = soup.find_all("a", class_= "dashboard-article-link")


for i in range(0,len(infotable)):
    CallDesc.append(infotable[i].getText().encode('utf-8'))
    CallLink.append(infotable[i]['href'])